# Helmholtz solver using Burton Miller formulation

https://github.com/bempp/bempp-cl/blob/main/notebooks/helmholtz/helmholtz_combined_exterior.ipynb

In [ ]:
import sys
sys.path.append('../build')

from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import Draw
from libbem import *
from ngsolve import Projector, Preconditioner
from ngsolve.krylovspace import CG, GMRes

In [ ]:
order=4

In [ ]:
# sp1 = Sphere( (0,0,0), 1)
screen = WorkPlane(Axes( (0,0,-3.5), Z, X)).RectangleC(20,20).Face()
sp = Fuse(Sphere( (0,0,0), 3).faces)
screen.faces.name="screen"
sp.faces.name="sphere"
shape = screen+sp
# Draw (shape)
mesh = Mesh( OCCGeometry(shape).GenerateMesh(maxh=0.5)).Curve(order)
Draw (mesh);

In [ ]:
fes_sphere = Compress(SurfaceL2(mesh, order=order, dual_mapping=False, complex=True, definedon=mesh.Boundaries("sphere")))
fes_sphere_test = Compress(SurfaceL2(mesh, order=order, dual_mapping=False, complex=True, definedon=mesh.Boundaries("sphere")))
fes_screen = Compress(SurfaceL2(mesh, order=order, dual_mapping=True, complex=True, definedon=mesh.Boundaries("screen")))
print ("ndof_sphere = ", fes_sphere.ndof, "ndof_screen =", fes_screen.ndof)

In [ ]:
kappa = 10
opt = dict (intorder=16, leafsize=40, eta=3., eps=1e-10, method="svd", testhmatrix=False)

In [ ]:
# SetTestoutFile("bem.out")
with TaskManager():
    V = HelmholtzSingleLayerPotentialOperator(fes_sphere, fes_sphere_test, kappa=kappa, **opt)
    K = HelmholtzDoubleLayerPotentialOperator(fes_sphere, fes_sphere_test, kappa=kappa, **opt)
    u = fes_sphere.TrialFunction()
    v = fes_sphere_test.TestFunction()
    Id = BilinearForm(u*v*ds).Assemble()

In [ ]:
lhs = 0.5 * Id.mat + K.mat - 1j * kappa * V.mat
source = 1j * kappa * exp(1j * kappa * x) * (specialcf.normal(3)[0]-1)
rhs = LinearForm(source*v*ds).Assemble()

In [ ]:
gfu = GridFunction(fes_sphere)
pre = BilinearForm(u*v*ds, diagonal=True).Assemble().mat.Inverse(inverse="sparsecholesky")
with TaskManager():
    gfu.vec[:] = GMRes(A=lhs, b=rhs.vec, pre=pre, maxsteps=200)

In [ ]:
print ("res = ", Norm(lhs*gfu.vec-rhs.vec))
Draw (gfu, order=3, min=-30, max=30)

# to come: prostprocessing on screen

In [ ]:
xxx interupt run all

In [ ]:
u,v = fes_screen.TnT()
mass = BilinearForm(u*v*ds).Assemble()
V2 = HelmholtzSingleLayerPotentialOperator(fes_sphere, fes_screen, kappa=kappa, **opt)

In [ ]:
gf1 = GridFunction(fes_sphere)
gf2 = GridFunction(fes_screen)
gf1.Set (1, definedon=mesh.Boundaries(".*"))
gf2.vec.data = (mass.mat.Inverse()@V2.mat) * gf1.vec

In [ ]:
Draw (1e2*gf2, mesh, order=3, animate_complex=True, min=-1, max=1);

In [ ]:
print (Id.mat)

In [ ]:
for t in Timers():
    if "ngbem" in t["name"]:
        print (t)